In [1]:
%load_ext autoreload
%autoreload 2

from configs.RecursiveVPSDE.Markovian_fSin.recursive_Markovian_PostMeanScaledScore_fSin_T256_H05_tl_5data import get_config as get_config_PMS
from configs.RecursiveVPSDE.Markovian_fSin.recursive_Markovian_PostMeanScore_fSin_T256_H05_tl_5data import get_config as get_config_PM
from configs.RecursiveVPSDE.Markovian_fSin.recursive_Markovian_fSinWithPosition_T256_H05_tl_5data import get_config as get_config_TS

import numpy as np
import torch

import matplotlib.pyplot as plt

In [2]:
config_TS = get_config_TS()
config_PM = get_config_PM()
config_PMS = get_config_PMS()
assert(config_TS.beta_min == config_PM.beta_min == config_PMS.beta_min)
assert(config_TS.ts_length == config_PM.ts_length == config_PMS.ts_length)
Delta = 1./config_TS.ts_length

In [3]:
from src.classes.ClassFractionalSin import FractionalSin
from tqdm import tqdm
import time
td = []
S = 1000
T = 332
isUnitInterval = True
mean_rev = config_TS.mean_rev
diff = config_TS.diffusion
initial_state = 0.
rvs = None
H = config_TS.hurst
deltaT = 1./ 256
t0 = 0.
t1 = deltaT*T
fSin = FractionalSin(mean_rev=mean_rev, diff=diff, X0=initial_state)
for k in range(10):
    true_data = np.array(
        [fSin.euler_simulation(H=H, N=T, deltaT=deltaT, isUnitInterval=isUnitInterval, X0=0, Ms=None, gaussRvs=rvs,
                               t0=t0, t1=t1) for _ in tqdm(range(S))]).reshape(
        (S, T+1))
    assert (true_data.shape == (S, T+1))
    td.append(true_data)
    time.sleep(10)

100%|██████████| 1000/1000 [00:07<00:00, 136.88it/s]


In [10]:
S = 10*S
true_data = np.concatenate(td, axis=0)
assert (true_data.shape == (S, T+1))

(10000, 333) 10000 333


In [11]:
assert(config_TS.data_path == config_PM.data_path == config_PMS.data_path)
assert (config_TS.max_diff_steps == config_PM.max_diff_steps == config_PMS.max_diff_steps == 10000)
true_data = true_data[:, 1:]#np.load(config_TS.data_path)
T = true_data.shape[1]
print(true_data.shape)

(10000, 332)


In [12]:
def moment_calculations(data):
    from scipy.stats import moment
    try:
        data = data.numpy()
    except AttributeError as e:
        print(e)
        pass
    means = np.mean(data, axis=0)
    assert(means.shape[0] == data.shape[1])
    vars, skew, kurt = moment(data, moment=[2,3,4], axis=0)
    assert(means.shape == vars.shape == skew.shape == kurt.shape)
    return means, vars, skew, kurt

In [13]:
from scipy.stats import wasserstein_distance
import ot
def bootstrap_test_wasserstein(sample1, sample2, num_bootstrap=1000):
    # Step 1: Compute the observed Wasserstein distance
    M = ot.dist(sample1, sample2, metric='euclidean')
    a = np.ones((sample1.shape[0],)) / sample1.shape[0]  # Uniform weights for X
    b = np.ones((sample1.shape[0],)) / sample1.shape[0]  # Uniform weights for Y

    #observed_distance = wasserstein_distance(sample1, sample2)
    print(a.shape, b.shape, M.shape)
    observed_distance = ot.lp.emd2(a, b, M = M)
    # Step 2: Bootstrap samples
    bootstrap_distances = []
    combined = np.vstack([sample1, sample2])
    for _ in range(num_bootstrap):
        np.random.shuffle(combined)
        obs_perm = combined[:sample1.shape[0], :]
        sim_perm = combined[sample2.shape[0]:, :]

        # Step 3: Compute Wasserstein distance for bootstrap samples
        M = ot.dist(obs_perm, sim_perm, metric='euclidean')
        boot_distance = (ot.lp.emd2(a, b, M = M))
        #boot_distance = wasserstein_distance(boot_sample1, boot_sample2)
        bootstrap_distances.append(boot_distance)

    # Step 4: Calculate p-value and confidence intervals
    bootstrap_distances = np.array(bootstrap_distances)
    fig, ax = plt.subplots(figsize=(14, 9))
    counts, bins, patches  = ax.hist(bootstrap_distances, bins=150, alpha=0.75,density=True)
    ax.set_title("Histogram of Permutation-Test Wasserstein Distances", fontsize=20)
    ax.axvline(observed_distance, 0, np.max(counts), color="orange", label="Observed Wasserstein Distance")
    ax.set_xlabel("Wasserstein Distances", fontsize=20)
    ax.set_ylabel("Normalised Count", fontsize=20)
    ax.legend(fontsize=18)
    ax.tick_params(labelsize=18)
    plt.savefig("/Users/marcos/Library/CloudStorage/OneDrive-ImperialCollegeLondon/StatML_CDT/Year2/DiffusionModelPresentationImages/wassersteindistances.png")
    #plt.show()
    plt.close()
    p_value = np.mean(bootstrap_distances >= observed_distance)
    #print(bootstrap_distances, observed_distance, p_value)

    return observed_distance, p_value

In [14]:
def plot_ts(ax, ts, T, label, moment):
    time_axis = np.linspace(Delta, Delta*T, T)
    plt.plot(time_axis, ts, label=f"{label}")
    ax.set_title(f"Empirical {moment}", fontsize=40)
    ax.tick_params(labelsize=38)
    ax.legend(fontsize=25)

In [15]:
num_paths = 1000

In [16]:
true_m, true_v, true_s, true_k = moment_calculations(true_data[np.random.choice(np.arange(true_data.shape[0]), num_paths),:])

'numpy.ndarray' object has no attribute 'numpy'


In [19]:
Nepoch = 960
means = [true_m]
vars = [true_v]
skews = [true_s]
kurt = [true_k]
ppaths = [true_data]
for config in [config_TS]:
    Ndiff = 10000
    for es in [0,5,10]:#[0, 5, 10, 20, 30,40,50,60,70,80,100, 150, 200]
        if "PMS" in config.scoreNet_trained_path:
            type = "PMS"
        elif "PM" in config.scoreNet_trained_path:
            type = "PM"
        else:
            type = ""
        try:
            file_path = f"/Users/marcos/GitHubRepos/FractionalBrownianMotion/experiments/results/TS{type}_ES{es}_fSin_PathGen_{Nepoch}Nep_{config.loss_factor}LFactor_10MeanRev_{Ndiff}DiffSteps_"#{type}_mkv_ES
            if type == "":
                type="Standard"

            paths = torch.load(file_path+"paths")
            paths = paths[np.random.choice(np.arange(paths.shape[0]), num_paths), 1:T+1]
            sampled_true_data = true_data[np.random.choice(np.arange(true_data.shape[0]), paths.shape[0]),:]
            print(f"ES {es} with p val {bootstrap_test_wasserstein(paths.numpy(), sampled_true_data)[1]}\n")
            print(f"ES {es} with p val {bootstrap_test_wasserstein(paths.numpy()[:,:256], sampled_true_data[:,:256])[1]}\n")
            ppaths.append(paths)
            print(type, es)
            if type == "":
                type="Standard"
            m, v, k, s = moment_calculations(paths)
            means.append(m)
            vars.append(v)
            skews.append(s)
            kurt.append(k)
        except FileNotFoundError as e:
            ppaths.append(np.zeros((num_paths, T)))
            m = np.zeros(T)
            means.append(m)
            vars.append(m)
            skews.append(m)
            kurt.append(m)
            print(e)

for t in range(0,ppaths[0].shape[1], 50):
    for i in range(0):
        print(len(means))
        if i == 0:
            label = "True"
            #plt.hist(ppaths[i][:, t], bins=150, density=True, alpha=0.75, label=label)
        elif i == 1:
            label = f"{type} ES 0"
        elif i == 2:
            label = f"{type} ES 5"
        elif i == 3:
            label = f"{type} ES 10"
        elif i == 4:
            label = f"{type} ES 20"
            #plt.hist(ppaths[i][:, t], bins=150, density=True, alpha=0.75, label=label)
        elif i ==5:
            label = f"{type} ES 50"
            #plt.hist(ppaths[i][:, t], bins=150, density=True, alpha=0.75, label=label)
        elif i == 6:
            label=f"{type} ES 100"
            #plt.hist(ppaths[i][:, t], bins=150, density=True, alpha=0.75, label=label)
        elif i == 7:
            label=f"{type} ES 150"
        elif i == 8:
            label=f"{type} ES 200"


    plt.title(f"Histogram at time index {t}")
    plt.legend()
    #plt.show()
    plt.close()

for mom in ["Mean", "Variance", "Skew", "Kurtosis"]:
    print(len(means))
    if mom == "Mean":
        ts = means
    elif mom == "Variance":
        ts = vars
    elif mom == "Skew":
        ts = skews
    else:
        ts = kurt
    fig, ax = plt.subplots(figsize=(14,9))
    for i in range(0):
        if i == 0:
            label = "True"
        elif i == 1:
            label = f"{type} ES 0"
        elif i == 2:
            label = f"{type} ES 5"
        elif i == 3:
            label = f"{type} ES 10"
        elif i == 4:
            label = f"{type} ES 20"
        elif i ==5:
            label = f"{type} ES 50"
        elif i == 6:
            label=f"{type} ES 100"
        elif i == 7:
            label=f"{type} ES 150"
        elif i == 8:
            label=f"{type} ES 200"
        #plot_ts(ax=ax, ts=ts[i], T=T,label=f"{label} {mom}", moment=f"{mom}")
    #plt.show()
    plt.close()




(1000,) (1000,) (1000, 1000)
ES 0 with p val 0.001

(1000,) (1000,) (1000, 1000)
ES 0 with p val 0.001

Standard 0
(1000,) (1000,) (1000, 1000)
ES 5 with p val 0.007

(1000,) (1000,) (1000, 1000)
ES 5 with p val 0.004

Standard 5
(1000,) (1000,) (1000, 1000)
ES 10 with p val 0.009

(1000,) (1000,) (1000, 1000)


No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.
No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.
No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.
No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.
No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.
No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.
No artists with labels found to put in legend.  Note that 

ES 10 with p val 0.003

Standard 10
4
4
4
4


In [22]:
Nepoch = 960
means = [true_m]
vars = [true_v]
skews = [true_s]
kurt = [true_k]
ppaths = [true_data]
for config in [config_PM]:
    Ndiff = 10000
    for es in [20, 30, 40, 50, 60, 70, 80]:#[0, 5, 10, 20, 30,40,50,60,70,80,100, 150, 200]
        if "PMS" in config.scoreNet_trained_path:
            type = "PMS"
        elif "PM" in config.scoreNet_trained_path:
            type = "PM"
        else:
            type = ""
        try:
            file_path = f"/Users/marcos/GitHubRepos/FractionalBrownianMotion/experiments/results/TS{type}_ES{es}_fSin_PathGen_{Nepoch}Nep_{config.loss_factor}LFactor_10MeanRev_{Ndiff}DiffSteps_"#{type}_mkv_ES
            if type == "":
                type="Standard"

            paths = torch.load(file_path+"paths")
            paths = paths[np.random.choice(np.arange(paths.shape[0]), num_paths), 1:T+1]
            sampled_true_data = true_data[np.random.choice(np.arange(true_data.shape[0]), paths.shape[0]),:]
            print(f"ES {es} with p val {bootstrap_test_wasserstein(paths.numpy(), sampled_true_data)[1]}\n")
            print(f"ES {es} with p val {bootstrap_test_wasserstein(paths.numpy()[:,:256], sampled_true_data[:,:256])[1]}\n")
            ppaths.append(paths)
            print(type, es)
            if type == "":
                type="Standard"
            m, v, k, s = moment_calculations(paths)
            means.append(m)
            vars.append(v)
            skews.append(s)
            kurt.append(k)
        except FileNotFoundError as e:
            ppaths.append(np.zeros((num_paths, T)))
            m = np.zeros(T)
            means.append(m)
            vars.append(m)
            skews.append(m)
            kurt.append(m)
            print(e)

for t in range(0,ppaths[0].shape[1], 50):
    for i in range(0):
        print(len(means))
        if i == 0:
            label = "True"
            #plt.hist(ppaths[i][:, t], bins=150, density=True, alpha=0.75, label=label)
        elif i == 1:
            label = f"{type} ES 0"
        elif i == 2:
            label = f"{type} ES 5"
        elif i == 3:
            label = f"{type} ES 10"
        elif i == 4:
            label = f"{type} ES 20"
            #plt.hist(ppaths[i][:, t], bins=150, density=True, alpha=0.75, label=label)
        elif i ==5:
            label = f"{type} ES 50"
            #plt.hist(ppaths[i][:, t], bins=150, density=True, alpha=0.75, label=label)
        elif i == 6:
            label=f"{type} ES 100"
            #plt.hist(ppaths[i][:, t], bins=150, density=True, alpha=0.75, label=label)
        elif i == 7:
            label=f"{type} ES 150"
        elif i == 8:
            label=f"{type} ES 200"


    plt.title(f"Histogram at time index {t}")
    plt.legend()
    #plt.show()
    plt.close()

for mom in ["Mean", "Variance", "Skew", "Kurtosis"]:
    print(len(means))
    if mom == "Mean":
        ts = means
    elif mom == "Variance":
        ts = vars
    elif mom == "Skew":
        ts = skews
    else:
        ts = kurt
    fig, ax = plt.subplots(figsize=(14,9))
    for i in range(0):
        if i == 0:
            label = "True"
        elif i == 1:
            label = f"{type} ES 0"
        elif i == 2:
            label = f"{type} ES 5"
        elif i == 3:
            label = f"{type} ES 10"
        elif i == 4:
            label = f"{type} ES 20"
        elif i ==5:
            label = f"{type} ES 50"
        elif i == 6:
            label=f"{type} ES 100"
        elif i == 7:
            label=f"{type} ES 150"
        elif i == 8:
            label=f"{type} ES 200"
        #plot_ts(ax=ax, ts=ts[i], T=T,label=f"{label} {mom}", moment=f"{mom}")
    #plt.show()
    plt.close()




(1000,) (1000,) (1000, 1000)
ES 20 with p val 0.0

(1000,) (1000,) (1000, 1000)
ES 20 with p val 0.0

PM 20
(1000,) (1000,) (1000, 1000)
ES 30 with p val 0.001

(1000,) (1000,) (1000, 1000)
ES 30 with p val 0.0

PM 30
(1000,) (1000,) (1000, 1000)
ES 40 with p val 0.002

(1000,) (1000,) (1000, 1000)
ES 40 with p val 0.003

PM 40
(1000,) (1000,) (1000, 1000)
ES 50 with p val 0.0

(1000,) (1000,) (1000, 1000)
ES 50 with p val 0.001

PM 50
(1000,) (1000,) (1000, 1000)
ES 60 with p val 0.0

(1000,) (1000,) (1000, 1000)
ES 60 with p val 0.0

PM 60
(1000,) (1000,) (1000, 1000)
ES 70 with p val 0.0

(1000,) (1000,) (1000, 1000)
ES 70 with p val 0.0

PM 70
(1000,) (1000,) (1000, 1000)
ES 80 with p val 0.0

(1000,) (1000,) (1000, 1000)


No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.
No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.
No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.
No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.
No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.
No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.
No artists with labels found to put in legend.  Note that 

ES 80 with p val 0.001

PM 80
8
8
8
8


In [24]:
Nepoch = 960
means = [true_m]
vars = [true_v]
skews = [true_s]
kurt = [true_k]
ppaths = [true_data]
for config in [config_PMS]:
    Ndiff = 10000
    for es in [0,5,10]:#[0, 5, 10, 20, 30,40,50,60,70,80,100, 150, 200]
        if "PMS" in config.scoreNet_trained_path:
            type = "PMS"
        elif "PM" in config.scoreNet_trained_path:
            type = "PM"
        else:
            type = ""
        try:
            file_path = f"/Users/marcos/GitHubRepos/FractionalBrownianMotion/experiments/results/TS{type}_ES{es}_fSin_PathGen_{Nepoch}Nep_{config.loss_factor}LFactor_10MeanRev_{Ndiff}DiffSteps_"#{type}_mkv_ES
            if type == "":
                type="Standard"

            paths = torch.load(file_path+"paths")
            paths = paths[np.random.choice(np.arange(paths.shape[0]), num_paths), 1:T+1]
            sampled_true_data = true_data[np.random.choice(np.arange(true_data.shape[0]), paths.shape[0]),:]
            print(f"ES {es} with p val {bootstrap_test_wasserstein(paths.numpy(), sampled_true_data)[1]}\n")
            print(f"ES {es} with p val {bootstrap_test_wasserstein(paths.numpy()[:,:256], sampled_true_data[:,:256])[1]}\n")
            ppaths.append(paths)
            print(type, es)
            if type == "":
                type="Standard"
            m, v, k, s = moment_calculations(paths)
            means.append(m)
            vars.append(v)
            skews.append(s)
            kurt.append(k)
        except FileNotFoundError as e:
            ppaths.append(np.zeros((num_paths, T)))
            m = np.zeros(T)
            means.append(m)
            vars.append(m)
            skews.append(m)
            kurt.append(m)
            print(e)

for t in range(0,ppaths[0].shape[1], 50):
    for i in range(0):
        print(len(means))
        if i == 0:
            label = "True"
            #plt.hist(ppaths[i][:, t], bins=150, density=True, alpha=0.75, label=label)
        elif i == 1:
            label = f"{type} ES 0"
        elif i == 2:
            label = f"{type} ES 5"
        elif i == 3:
            label = f"{type} ES 10"
        elif i == 4:
            label = f"{type} ES 20"
            #plt.hist(ppaths[i][:, t], bins=150, density=True, alpha=0.75, label=label)
        elif i ==5:
            label = f"{type} ES 50"
            #plt.hist(ppaths[i][:, t], bins=150, density=True, alpha=0.75, label=label)
        elif i == 6:
            label=f"{type} ES 100"
            #plt.hist(ppaths[i][:, t], bins=150, density=True, alpha=0.75, label=label)
        elif i == 7:
            label=f"{type} ES 150"
        elif i == 8:
            label=f"{type} ES 200"


    plt.title(f"Histogram at time index {t}")
    plt.legend()
    #plt.show()
    plt.close()

for mom in ["Mean", "Variance", "Skew", "Kurtosis"]:
    print(len(means))
    if mom == "Mean":
        ts = means
    elif mom == "Variance":
        ts = vars
    elif mom == "Skew":
        ts = skews
    else:
        ts = kurt
    fig, ax = plt.subplots(figsize=(14,9))
    for i in range(0):
        if i == 0:
            label = "True"
        elif i == 1:
            label = f"{type} ES 0"
        elif i == 2:
            label = f"{type} ES 5"
        elif i == 3:
            label = f"{type} ES 10"
        elif i == 4:
            label = f"{type} ES 20"
        elif i ==5:
            label = f"{type} ES 50"
        elif i == 6:
            label=f"{type} ES 100"
        elif i == 7:
            label=f"{type} ES 150"
        elif i == 8:
            label=f"{type} ES 200"
        #plot_ts(ax=ax, ts=ts[i], T=T,label=f"{label} {mom}", moment=f"{mom}")
    #plt.show()
    plt.close()




(1000,) (1000,) (1000, 1000)
ES 0 with p val 0.0

(1000,) (1000,) (1000, 1000)
ES 0 with p val 0.0

PMS 0
(1000,) (1000,) (1000, 1000)
ES 5 with p val 0.0

(1000,) (1000,) (1000, 1000)
ES 5 with p val 0.0

PMS 5
(1000,) (1000,) (1000, 1000)
ES 10 with p val 0.0

(1000,) (1000,) (1000, 1000)


No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.
No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.
No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.
No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.
No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.
No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.
No artists with labels found to put in legend.  Note that 

ES 10 with p val 0.0

PMS 10
4
4
4
4
